In [1]:
import os
import csv
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps
import math
from calendar import monthrange
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
from edc_base.utils import get_utcnow, age

edc_protocol = django_apps.get_app_config('edc_protocol')

In [2]:
##
# Get holidays loaded on the system for a specified month, and year.
##
def get_holidays(year, month):
    facility_app_config = django_apps.get_app_config('edc_facility')

    facility = facility_app_config.get_facility('5-day clinic')

    holiday_list = facility.holidays.holidays.filter(
        local_date__year=year,
        local_date__month=month).values_list('local_date', flat=True)
    return [h.day for h in holiday_list]

In [3]:
##
# Query the participant note for pids already scheduled for a follow up.
##
def get_pids_scheduled():
    return ParticipantNote.objects.filter(
        title='Follow Up Schedule', ).values_list('subject_identifier', flat=True)

In [4]:
## 
# Returns last date a participant was scheduled for a follow,
# by querying the participants' note by latest scheduled.
##
def get_last_date_fu_scheduled(max_per_day):
    part_notes = ParticipantNote.objects.filter(title='Follow Up Schedule', )
    if part_notes:
        last_scheduled = part_notes.latest('date')
        num_scheduled = part_notes.filter(date=last_scheduled.date)
        return last_scheduled.date, max_per_day - num_scheduled.count()
    return None, 0

In [5]:
## 
# Get all the days in a year available to schedule a follow, based
# on a particular start date.
##
def get_dates_to_schedule(start_date=None, available_dates=[]):
    for month in range(start_date.month, 13): # Month is always 1..12
        holidays = get_holidays(start_date.year, month)
        initial_day = start_date.day if start_date.month == month else 1
        for day in range(initial_day, monthrange(start_date.year, month)[1] + 1):
            current_date = date(start_date.year, month, day)
            if day not in holidays and current_date.weekday() < 5:
                available_dates.append(current_date)
    return available_dates

In [6]:
## 
# Returns the number of available slots to schedule partipants for a given day,
# this is with regards to the max number that can be scheduled per day.
##
def slots_available(fu_date, max_per_day):
    fu_notes = ParticipantNote.objects.filter(title='Follow Up Schedule', date=fu_date).count()
    return max_per_day - fu_notes

In [7]:
## 
# Returns the number of days needed to schedule all participants' in a list,
# this is with regards to the max number that can be scheduled per day.
##
def days_needed(pids_count, max_per_day, remainder_slots):
    days_needed = 0
    if remainder_slots:
        days_needed = 1
        pids_count - remainder_slots
    days_needed += pids_count/max_per_day if not pids_count%max_per_day else pids_count/max_per_day + 1
    return days_needed

In [8]:
## 
# Updates a dates key dict with a list of pid values to be scheduled for that date.
##
def schedule_for_date(schedule, day, pids, slot_nums):
    schedule.update({f'{day}': pids[:slot_nums]})
    del pids[:slot_nums]

In [9]:
def schedule_fu_visit(max_per_day, pids, begin_date=None, schedule = {}):
    start_date, remainder_slot = None, None
    if begin_date:
        start_date, remainder_slots = begin_date, slots_available(
            begin_date, max_per_day)
    else:
        start_date, remainder_slots = get_last_date_fu_scheduled(max_per_day)

    full_scheduled = remainder_slots == 0
    if start_date and full_scheduled:
        start_date = start_date + timedelta(days=1)

    days_available = get_dates_to_schedule(start_date=start_date, )

    last_day = begin_date
    while len(days_available) > 0 and pids:
        last_day = days_available[0]
        num_available = slots_available(last_day, max_per_day)
        if num_available:
            schedule_for_date(schedule, last_day, pids, num_available)
        del days_available[:1]
 
    if pids:
        schedule_fu_visit(max_per_day, pids, last_day, schedule)

    return schedule

In [11]:
# Child cohort A participants' ANC {45days before 18months and 45 days after}
child_identifiers=OnScheduleChildCohortABirth.objects.exclude(
    subject_identifier__in=get_pids_scheduled()).values_list('subject_identifier', flat=True)
childa_dob_limit = get_utcnow() - relativedelta(months=18)
childa_dob_lower_limit = childa_dob_limit - relativedelta(days=45)
childa_dob_upper_limit = childa_dob_limit + relativedelta(days=45)
cohort_anc = CaregiverChildConsent.objects.filter(cohort='cohort_a',
                                                  subject_identifier__in=child_identifiers, )
#                                                   child_dob__gte=childa_dob_lower_limit.date(),
#                                                   child_dob__gte=childa_dob_limit.date(),
#                                                   child_dob__lte=childa_dob_upper_limit.date()
ageing_out = []
childa_dob_lower_limit = childa_dob_lower_limit.date()
childa_dob_upper_limit = childa_dob_upper_limit.date()
for child in cohort_anc:
    child_age = age(child.child_dob, get_utcnow())
    lower_age = age(childa_dob_lower_limit, get_utcnow())
    upper_age = age(childa_dob_upper_limit, get_utcnow())
    if ((lower_age.years + lower_age.months/12) < (child_age.years + child_age.months/12)
            and (upper_age.years + upper_age.months/12) > (child_age.years + child_age.months/12)):
        ageing_out.append(child)
len(ageing_out)

0

In [12]:
# Child cohort A participants' PRIOR-BHP
a_records = {}
a_urgent = []
exclude_pids = list(child_identifiers)
exclude_pids.extend(list(get_pids_scheduled()))
childa_dob_upper_limit = get_utcnow() - relativedelta(years=5)
cohort_a_prior = CaregiverChildConsent.objects.exclude(
    subject_identifier__in=exclude_pids).filter(cohort='cohort_a',
                                                child_dob__gte=childa_dob_upper_limit.date())

for subject in cohort_a_prior:
    child_age = age(subject.child_dob, get_utcnow())
    age_str = f'{child_age.years}.{child_age.months}'
    a_records.update({f'{subject.subject_identifier}': (child_age.years + child_age.months/12)})
    if (child_age.years + child_age.months/12) >= (4 + 5/12):
        a_urgent.append({'subject_identifier': subject.subject_identifier, 'age': age_str})

# sort in order of priority
sorted_arecords = sorted(a_records.items(), key=lambda record: record[1], reverse=True)
sorted_qs = cohort_a_prior.order_by('child_dob').values_list('subject_identifier', flat=True)

schedule = schedule_fu_visit(1, list(sorted_qs), get_utcnow())
len(sorted_arecords)

88

In [14]:
# Add PIDs to calendar
# TO DO: consider both cohorts before adding to calendar
created = 0
for fu_date, sidxs in schedule.items():
    for sidx in sidxs:
        ParticipantNote.objects.create(
            subject_identifier=sidx, title='Follow Up Schedule', date=fu_date, color='grey')
    created += 1
print(created)

89


In [27]:
# Export to csv
import csv
keys = ['subject_identifier', 'age']
with open('cohort_a_urgent.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(a_urgent)

In [17]:
# Child cohort B participants'
b_records = {}
b_urgent = []
childb_dob_upper_limit = get_utcnow() - relativedelta(years=10, months=5)
childb_dob_lower_limit = get_utcnow() - relativedelta(years=5)
cohort_b = CaregiverChildConsent.objects.exclude(
    subject_identifier__in=get_pids_scheduled()).filter(cohort='cohort_b',
                                                        child_dob__lte=childb_dob_lower_limit.date(),
                                                        child_dob__gt=childb_dob_upper_limit.date())

for subject in cohort_b:
    child_age = age(subject.child_dob, get_utcnow())
    age_str = f'{child_age.years}.{child_age.months}'
    b_records.update({f'{subject.subject_identifier}': age_str})
    if (child_age.years + child_age.months/12) >= 10 and (child_age.years + child_age.months/12) <= (10 + 5/12):
        b_urgent.append({'subject_identifier': subject.subject_identifier, 'age': age_str})

# sort in order of priority
sorted_brecords = sorted(b_records.items(), key=lambda record: record[1], reverse=True)
sorted_qs = cohort_b.order_by('child_dob').values_list('subject_identifier', flat=True)

b_schedule = schedule_fu_visit(2, list(sorted_qs), get_utcnow())
len(sorted_brecords)
sorted_qs

<QuerySet ['B142-040990065-0-25', 'B142-040990065-0-35', 'B142-040990280-5-10', 'B142-040990109-6-10', 'B142-040990077-5-10', 'B142-040990145-0-10', 'B142-040990966-9-10', 'B142-040990252-4-10', 'B142-040990248-2-10', 'B142-040990287-0-10', 'C142-040990124-5-10', 'B142-040990040-3-10', 'B142-040990643-4-10', 'B142-040990152-6-10', 'B142-040990081-7-10', 'B142-040990045-2-10', 'B142-040990221-9-10', 'B142-040990156-7-10', 'B142-040990324-1-10', 'B142-040990120-3-10', '...(remaining elements truncated)...']>

In [19]:
# Add PIDs to calendar
# TO DO: consider both cohorts before adding to calendar
created = 0
for fu_date, sidxs in b_schedule.items():
    for sidx in sidxs:
        ParticipantNote.objects.create(
            subject_identifier=sidx, title='Follow Up Schedule', date=fu_date, color='grey')
    created += 1
print(created)

151


In [ ]:
# Export to csv
import csv
keys = ['subject_identifier', 'age']
with open('cohort_b_urgent.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(b_urgent)

In [59]:
# Child cohort C participants'
c_records = {}
childc_dob_lower_limit = get_utcnow() - relativedelta(years=10)
cohort_c = CaregiverChildConsent.objects.exclude(
    subject_identifier__in=get_pids_scheduled()).filter(cohort='cohort_c',
                                                        child_dob__lte=childc_dob_lower_limit.date(), )
for subject in cohort_c:
    child_age = age(subject.child_dob, get_utcnow())
    age_str = f'{child_age.years}.{child_age.months}'
    c_records.update({f'{subject.subject_identifier}': age_str})
cohort_c.count()

78

194